In [8]:
import numpy as np
from collections import Counter
from collections import defaultdict
from itertools import permutations
import time
import math

In [9]:
def read_data(file_path):
    with open(file_path,'r') as f:
        data = f.readlines()
    data = [item.rstrip('\n').split(' ') for item in data]
    s_data = []
    new_data = []
    for d in data:
        if d[0]:
            new_data.append(d)
        else:
            s_data.append(new_data)
            new_data = []
    return s_data

def read_template(template_path):
    with open(template_path) as f:
        template = f.readlines()
    template = [t.rstrip('\n') for t in template if t[0] in ['U','B']]
    return template

In [10]:
train_data

NameError: name 'train_data' is not defined

In [11]:
data_file_path = r'data\small_train.data'
train_data = read_data(data_file_path)
template_path = r'data\template'
template = read_template(template_path)

### features

In [12]:
START_LABEL = '@@'
START_LABEL_IDX = 0
STOP_LABEL = '##'
SEP1 = '^^'
SEP2 = '%%'

class Feature():
    def __init__(self, data):
        self.X = data
        self.labels = Counter()
        self.feats = Counter()
        self.feats2idx = dict()
        self.labels2idx = dict()
        self.feats_size, self.labels_size = 0, 0
        
        self.label_dic = {START_LABEL: START_LABEL_IDX}
        self.label_array = [START_LABEL]
        self.num_features = 0
        self.feature_dic = dict()
        self.observation_set = set()
        self.empirical_counts = Counter()
        #self.valid_featfunc_in_data = Counter()
    
    
    def get_sentence_feats_labels(self, sent):
        feats, labels = [], []
        featfunc_in_sent = defaultdict(int)
        sent_len = len(sent)
        featfunc_in_sent_position = defaultdict(list)
        for idx_w, w in enumerate(sent):
            labels.append(w[-1])
            feats.append('U02:%s' % w[0])
            feats.append('U12:%s' % w[1])
            ###
            featfunc_in_sent['U02:%s' % w[0] + SEP1 + w[-1]] += 1
            featfunc_in_sent['U12:%s' % w[1] + SEP1 + w[-1]] += 1
            featfunc_in_sent_position['U02:%s' % w[0] + SEP1 + w[-1]].append(idx_w)
            featfunc_in_sent_position['U12:%s' % w[1] + SEP1 + w[-1]].append(idx_w)
            if idx_w>0:
                if idx_w < sent_len-1:
                    featfunc_in_sent['U02:%s' % w[0] + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]] += 1
                    featfunc_in_sent['U12:%s' % w[1] + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]] += 1
                    featfunc_in_sent_position['U02:%s' % w[0] + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]].append(idx_w)
                    featfunc_in_sent_position['U12:%s' % w[1] + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]].append(idx_w)
                else:
                    featfunc_in_sent['U02:%s' % w[0] + SEP1 + w[-1] + SEP2 + STOP_LABEL] += 1
                    featfunc_in_sent['U12:%s' % w[1] + SEP1 + w[-1] + SEP2 + STOP_LABEL] += 1
                    featfunc_in_sent_position['U02:%s' % w[0] + SEP1 + w[-1] + SEP2 + STOP_LABEL].append(idx_w)
                    featfunc_in_sent_position['U12:%s' % w[1] + SEP1 + w[-1] + SEP2 + STOP_LABEL].append(idx_w)
                    
            else:
                featfunc_in_sent['U02:%s' % w[0] + SEP1 + START_LABEL + SEP2 + w[-1]] += 1
                featfunc_in_sent['U12:%s' % w[1] + SEP1 + START_LABEL + SEP2 + w[-1]] += 1
                featfunc_in_sent_position['U02:%s' % w[0] + SEP1 + START_LABEL + SEP2 + w[-1]].append(idx_w)
                featfunc_in_sent_position['U12:%s' % w[1] + SEP1 + START_LABEL + SEP2 + w[-1]].append(idx_w)
            
            if idx_w < sent_len-1:
                feats.append('U03:%s' % sent[idx_w+1][0])
                feats.append('U13:%s' % sent[idx_w+1][1])
                feats.append('U06:%s/%s' % (w[0], sent[idx_w+1][0]))
                feats.append('U17:%s/%s' % (w[1], sent[idx_w+1][1]))
                
                featfunc_in_sent['U03:%s' % sent[idx_w+1][0] + SEP1 + w[-1]] += 1
                featfunc_in_sent['U13:%s' % sent[idx_w+1][1] + SEP1 + w[-1]] += 1
                featfunc_in_sent['U06:%s/%s' % (w[0], sent[idx_w+1][0]) + SEP1 + w[-1]] += 1
                featfunc_in_sent['U17:%s/%s' % (w[1], sent[idx_w+1][1]) + SEP1 + w[-1]] += 1
                featfunc_in_sent_position['U03:%s' % sent[idx_w+1][0] + SEP1 + w[-1]].append(idx_w)
                featfunc_in_sent_position['U13:%s' % sent[idx_w+1][1] + SEP1 + w[-1]].append(idx_w)
                featfunc_in_sent_position['U06:%s/%s' % (w[0], sent[idx_w+1][0]) + SEP1 + w[-1]].append(idx_w)
                featfunc_in_sent_position['U17:%s/%s' % (w[1], sent[idx_w+1][1]) + SEP1 + w[-1]].append(idx_w)
                
                if idx_w < sent_len-2:
                    feats.append('U04:%s' % sent[idx_w+2][0])
                    feats.append('U14:%s' % sent[idx_w+2][1])
                    feats.append('U18:%s/%s' % (sent[idx_w+1][1],sent[idx_w+2][1]))
                    feats.append('U22:%s/%s/%s' % (w[1],sent[idx_w+1][1],sent[idx_w+2][1]))
                    
                    featfunc_in_sent['U04:%s' % sent[idx_w+2][0] + SEP1 + w[-1]] += 1
                    featfunc_in_sent['U14:%s' % sent[idx_w+2][1] + SEP1 + w[-1]] += 1
                    featfunc_in_sent['U18:%s/%s' % (sent[idx_w+1][1],sent[idx_w+2][1]) + SEP1 + w[-1]] += 1
                    featfunc_in_sent['U22:%s/%s/%s' % (w[1],sent[idx_w+1][1],sent[idx_w+2][1]) + SEP1 + w[-1]] += 1
                    featfunc_in_sent_position['U04:%s' % sent[idx_w+2][0] + SEP1 + w[-1]].append(idx_w)
                    featfunc_in_sent_position['U14:%s' % sent[idx_w+2][1] + SEP1 + w[-1]].append(idx_w)
                    featfunc_in_sent_position['U18:%s/%s' % (sent[idx_w+1][1],sent[idx_w+2][1]) + SEP1 + w[-1]].append(idx_w)
                    featfunc_in_sent_position['U22:%s/%s/%s' % (w[1],sent[idx_w+1][1],sent[idx_w+2][1]) + SEP1 + w[-1]].append(idx_w)
                    
                    
                if idx_w > 0:
                    feats.append('U21:%s/%s/%s' % (sent[idx_w-1][1],w[1],sent[idx_w+1][1]))
                    
                    featfunc_in_sent['U21:%s/%s/%s' % (sent[idx_w-1][1],w[1],sent[idx_w+1][1]) + SEP1 + w[-1]] += 1
                    featfunc_in_sent['U21:%s/%s/%s' % (sent[idx_w-1][1],w[1],sent[idx_w+1][1]) + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]] += 1
                    featfunc_in_sent_position['U21:%s/%s/%s' % (sent[idx_w-1][1],w[1],sent[idx_w+1][1]) + SEP1 + w[-1]].append(idx_w)
                    featfunc_in_sent_position['U21:%s/%s/%s' % (sent[idx_w-1][1],w[1],sent[idx_w+1][1]) + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]].append(idx_w)
                    
                    featfunc_in_sent['U03:%s' % sent[idx_w+1][0] + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]] += 1
                    featfunc_in_sent['U13:%s' % sent[idx_w+1][1] + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]] += 1
                    featfunc_in_sent['U06:%s/%s' % (w[0], sent[idx_w+1][0]) + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]] += 1
                    featfunc_in_sent['U17:%s/%s' % (w[1], sent[idx_w+1][1]) + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]] += 1
                    if idx_w < sent_len-2:
                        featfunc_in_sent['U04:%s' % sent[idx_w+2][0] + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]] +=1
                        featfunc_in_sent['U14:%s' % sent[idx_w+2][1] + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]] +=1
                        featfunc_in_sent['U18:%s/%s' % (sent[idx_w+1][1],sent[idx_w+2][1]) + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]] +=1
                        featfunc_in_sent['U22:%s/%s/%s' % (w[1],sent[idx_w+1][1],sent[idx_w+2][1]) + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]] +=1
                        featfunc_in_sent_position['U04:%s' % sent[idx_w+2][0] + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]].append(idx_w)
                        featfunc_in_sent_position['U14:%s' % sent[idx_w+2][1] + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]].append(idx_w)
                        featfunc_in_sent_position['U18:%s/%s' % (sent[idx_w+1][1],sent[idx_w+2][1]) + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]].append(idx_w)
                        featfunc_in_sent_position['U22:%s/%s/%s' % (w[1],sent[idx_w+1][1],sent[idx_w+2][1]) + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]].append(idx_w)
                        
                        
                    featfunc_in_sent_position['U03:%s' % sent[idx_w+1][0] + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]].append(idx_w)
                    featfunc_in_sent_position['U13:%s' % sent[idx_w+1][1] + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]].append(idx_w)
                    featfunc_in_sent_position['U06:%s/%s' % (w[0], sent[idx_w+1][0]) + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]].append(idx_w)
                    featfunc_in_sent_position['U17:%s/%s' % (w[1], sent[idx_w+1][1]) + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]].append(idx_w)

                else:
                    featfunc_in_sent['U03:%s' % sent[idx_w+1][0] + SEP1 + START_LABEL + SEP2 + w[-1]] += 1
                    featfunc_in_sent['U13:%s' % sent[idx_w+1][1] + SEP1 + START_LABEL + SEP2 +  w[-1]] += 1
                    featfunc_in_sent['U06:%s/%s' % (w[0], sent[idx_w+1][0]) + SEP1 + START_LABEL + SEP2 +  w[-1]] += 1
                    featfunc_in_sent['U17:%s/%s' % (w[1], sent[idx_w+1][1]) + SEP1 + START_LABEL + SEP2 +  w[-1]] += 1
                    featfunc_in_sent['U04:%s' % sent[idx_w+2][0] + SEP1 + START_LABEL + SEP2 +  w[-1]] += 1
                    featfunc_in_sent['U14:%s' % sent[idx_w+2][1] + SEP1 + START_LABEL + SEP2 +  w[-1]] += 1
                    featfunc_in_sent['U18:%s/%s' % (sent[idx_w+1][1],sent[idx_w+2][1]) + SEP1 + START_LABEL + SEP2 +  w[-1]] += 1
                    featfunc_in_sent['U22:%s/%s/%s' % (w[1],sent[idx_w+1][1],sent[idx_w+2][1]) + SEP1 + START_LABEL + SEP2 +  w[-1]] += 1
                    
                    featfunc_in_sent_position['U03:%s' % sent[idx_w+1][0] + SEP1 + START_LABEL + SEP2 + w[-1]].append(idx_w)
                    featfunc_in_sent_position['U13:%s' % sent[idx_w+1][1] + SEP1 + START_LABEL + SEP2 +  w[-1]].append(idx_w)
                    featfunc_in_sent_position['U06:%s/%s' % (w[0], sent[idx_w+1][0]) + SEP1 + START_LABEL + SEP2 +  w[-1]].append(idx_w)
                    featfunc_in_sent_position['U17:%s/%s' % (w[1], sent[idx_w+1][1]) + SEP1 + START_LABEL + SEP2 +  w[-1]].append(idx_w)
                    featfunc_in_sent_position['U04:%s' % sent[idx_w+2][0] + SEP1 + START_LABEL + SEP2 +  w[-1]].append(idx_w)
                    featfunc_in_sent_position['U14:%s' % sent[idx_w+2][1] + SEP1 + START_LABEL + SEP2 +  w[-1]].append(idx_w)
                    featfunc_in_sent_position['U18:%s/%s' % (sent[idx_w+1][1],sent[idx_w+2][1]) + SEP1 + START_LABEL + SEP2 +  w[-1]].append(idx_w)
                    featfunc_in_sent_position['U22:%s/%s/%s' % (w[1],sent[idx_w+1][1],sent[idx_w+2][1]) + SEP1 + START_LABEL + SEP2 +  w[-1]].append(idx_w)
                    
                    
            if idx_w > 0:
                feats.append('U01:%s' % sent[idx_w-1][0])
                feats.append('U11:%s' % sent[idx_w-1][1])
                feats.append('U05:%s/%s' % (sent[idx_w-1][0],w[0]))
                feats.append('U16:%s/%s' % (sent[idx_w-1][1],w[1]))
                ###
                featfunc_in_sent['U01:%s' % sent[idx_w-1][0] + SEP1 + w[-1]] +=1
                featfunc_in_sent['U11:%s' % sent[idx_w-1][1] + SEP1 + w[-1]] +=1
                featfunc_in_sent['U05:%s/%s' % (sent[idx_w-1][0],w[0]) + SEP1 + w[-1]] +=1
                featfunc_in_sent['U16:%s/%s' % (sent[idx_w-1][1],w[1]) + SEP1 + w[-1]] +=1
                featfunc_in_sent['U01:%s' % sent[idx_w-1][0] + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]] +=1
                featfunc_in_sent['U11:%s' % sent[idx_w-1][1] + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]] +=1
                featfunc_in_sent['U05:%s/%s' % (sent[idx_w-1][0],w[0]) + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]] +=1
                featfunc_in_sent['U16:%s/%s' % (sent[idx_w-1][1],w[1]) + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]] +=1
                                              
                featfunc_in_sent_position['U01:%s' % sent[idx_w-1][0] + SEP1 + w[-1]].append(idx_w)
                featfunc_in_sent_position['U11:%s' % sent[idx_w-1][1] + SEP1 + w[-1]].append(idx_w)
                featfunc_in_sent_position['U05:%s/%s' % (sent[idx_w-1][0],w[0]) + SEP1 + w[-1]].append(idx_w)
                featfunc_in_sent_position['U16:%s/%s' % (sent[idx_w-1][1],w[1]) + SEP1 + w[-1]].append(idx_w)
                featfunc_in_sent_position['U01:%s' % sent[idx_w-1][0] + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]].append(idx_w)
                featfunc_in_sent_position['U11:%s' % sent[idx_w-1][1] + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]].append(idx_w)
                featfunc_in_sent_position['U05:%s/%s' % (sent[idx_w-1][0],w[0]) + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]].append(idx_w)
                featfunc_in_sent_position['U16:%s/%s' % (sent[idx_w-1][1],w[1]) + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]].append(idx_w)
                
                if idx_w==sent_len-1:
                    featfunc_in_sent['U01:%s' % sent[idx_w-1][0] + SEP1 + w[-1] + SEP2 + STOP_LABEL] +=1
                    featfunc_in_sent['U11:%s' % sent[idx_w-1][1] + SEP1 + w[-1] + SEP2 + STOP_LABEL] +=1
                    featfunc_in_sent['U05:%s/%s' % (sent[idx_w-1][0],w[0]) + SEP1 + w[-1] + SEP2 + STOP_LABEL] +=1
                    featfunc_in_sent['U16:%s/%s' % (sent[idx_w-1][1],w[1]) + SEP1 + w[-1] + SEP2 + STOP_LABEL] +=1
                    featfunc_in_sent_position['U01:%s' % sent[idx_w-1][0] + SEP1 + w[-1] + SEP2 + STOP_LABEL].append(idx_w)
                    featfunc_in_sent_position['U11:%s' % sent[idx_w-1][1] + SEP1 + w[-1] + SEP2 + STOP_LABEL].append(idx_w)
                    featfunc_in_sent_position['U05:%s/%s' % (sent[idx_w-1][0],w[0]) + SEP1 + w[-1] + SEP2 + STOP_LABEL].append(idx_w)
                    featfunc_in_sent_position['U16:%s/%s' % (sent[idx_w-1][1],w[1]) + SEP1 + w[-1] + SEP2 + STOP_LABEL].append(idx_w)
                
                if idx_w > 1:
                    feats.append('U00:%s' % sent[idx_w-2][0])
                    feats.append('U10:%s' % sent[idx_w-2][1])
                    feats.append('U15:%s/%s' % (sent[idx_w-2][1],sent[idx_w-1][1]))
                    feats.append('U20:%s/%s/%s' % (sent[idx_w-2][1],sent[idx_w-1][1],w[1]))
                    ###
                    featfunc_in_sent['U00:%s' % sent[idx_w-2][0] + SEP1 + w[-1]] +=1
                    featfunc_in_sent['U10:%s' % sent[idx_w-2][1] + SEP1 + w[-1]] +=1
                    featfunc_in_sent['U15:%s/%s' % (sent[idx_w-2][1],sent[idx_w-1][1]) + SEP1 + w[-1]] +=1
                    featfunc_in_sent['U20:%s/%s/%s' % (sent[idx_w-2][1],sent[idx_w-1][1],w[1]) + SEP1 + w[-1]] +=1
                    featfunc_in_sent['U00:%s' % sent[idx_w-2][0]  + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]] +=1
                    featfunc_in_sent['U10:%s' % sent[idx_w-2][1]  + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]] +=1
                    featfunc_in_sent['U15:%s/%s' % (sent[idx_w-2][1],sent[idx_w-1][1])  + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]] +=1
                    featfunc_in_sent['U20:%s/%s/%s' % (sent[idx_w-2][1],sent[idx_w-1][1],w[1]) + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]] +=1
                    featfunc_in_sent_position['U00:%s' % sent[idx_w-2][0] + SEP1 + w[-1]].append(idx_w)
                    featfunc_in_sent_position['U10:%s' % sent[idx_w-2][1] + SEP1 +  w[-1]].append(idx_w)
                    featfunc_in_sent_position['U15:%s/%s' % (sent[idx_w-2][1],sent[idx_w-1][1]) + SEP1 + w[-1]].append(idx_w)
                    featfunc_in_sent_position['U20:%s/%s/%s' % (sent[idx_w-2][1],sent[idx_w-1][1],w[1]) + SEP1 + w[-1]].append(idx_w)
                    featfunc_in_sent_position['U00:%s' % sent[idx_w-2][0] + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]].append(idx_w)
                    featfunc_in_sent_position['U10:%s' % sent[idx_w-2][1] + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]].append(idx_w)
                    featfunc_in_sent_position['U15:%s/%s' % (sent[idx_w-2][1],sent[idx_w-1][1]) + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]].append(idx_w)
                    featfunc_in_sent_position['U20:%s/%s/%s' % (sent[idx_w-2][1],sent[idx_w-1][1],w[1]) + SEP1 + sent[idx_w-1][-1] + SEP2 + w[-1]].append(idx_w)
                    if idx_w == sent_len-1:
                        featfunc_in_sent['U00:%s' % sent[idx_w-2][0] + SEP1 + w[-1] + SEP2 + STOP_LABEL] +=1
                        featfunc_in_sent['U10:%s' % sent[idx_w-2][1] + SEP1 + w[-1] + SEP2 + STOP_LABEL] +=1
                        featfunc_in_sent['U15:%s/%s' % (sent[idx_w-2][1],sent[idx_w-1][1]) + SEP1 + w[-1] + SEP2 + STOP_LABEL] +=1
                        featfunc_in_sent['U20:%s/%s/%s' % (sent[idx_w-2][1],sent[idx_w-1][1],w[1]) + SEP1 + w[-1] + SEP2 + STOP_LABEL] +=1
                                              
                        featfunc_in_sent_position['U00:%s' % sent[idx_w-2][0]+ SEP1+w[-1] + SEP2 + STOP_LABEL].append(idx_w)
                        featfunc_in_sent_position['U10:%s' % sent[idx_w-2][1]+ SEP1+w[-1] + SEP2 + STOP_LABEL].append(idx_w)
                        featfunc_in_sent_position['U15:%s/%s' % (sent[idx_w-2][1],sent[idx_w-1][1])+ SEP1 + w[-1] + SEP2 + STOP_LABEL].append(idx_w)
                        featfunc_in_sent_position['U20:%s/%s/%s' % (sent[idx_w-2][1],sent[idx_w-1][1],w[1])+ SEP1 + w[-1] + SEP2 + STOP_LABEL].append(idx_w)
                            
            
        return feats, labels, featfunc_in_sent, featfunc_in_sent_position
    
    def get_word_feats(self, sent, idx):
        sent_len = len(sent)
        feats = list()
        feats.append('U02:%s' % sent[idx][0])
        feats.append('U12:%s' % sent[idx][1])
        if idx < sent_len-1:
            feats.append('U03:%s' % sent[idx+1][0])
            feats.append('U13:%s' % sent[idx+1][1])
            feats.append('U06:%s/%s' % (sent[idx][0], sent[idx+1][0]))
            feats.append('U17:%s/%s' % (sent[idx][1], sent[idx+1][1]))
            if idx < sent_len-2:
                feats.append('U04:%s' % sent[idx+2][0])
                feats.append('U14:%s' % sent[idx+2][1])
                feats.append('U18:%s/%s' % (sent[idx+1][1],sent[idx+2][1]))
                feats.append('U22:%s/%s/%s' % (sent[idx][1],sent[idx+1][1],sent[idx+2][1]))
            if idx > 0:
                feats.append('U21:%s/%s/%s' % (sent[idx-1][1],sent[idx][1],sent[idx+1][1]))
        if idx > 0:
            feats.append('U01:%s' % sent[idx-1][0])
            feats.append('U11:%s' % sent[idx-1][1])
            feats.append('U05:%s/%s' % (sent[idx-1][0],sent[idx][0]))
            feats.append('U16:%s/%s' % (sent[idx-1][1],sent[idx][1]))        
            if idx > 1:
                feats.append('U00:%s' % sent[idx-2][0])
                feats.append('U10:%s' % sent[idx-2][1])
                feats.append('U15:%s/%s' % (sent[idx-2][1],sent[idx-1][1]))
                feats.append('U20:%s/%s/%s' % (sent[idx-2][1],sent[idx-1][1],sent[idx][1]))
        return feats
    
    def add(self, prev_y,y,sent,idx):
        for feature_string in self.get_word_feats(sent, idx):
            if feature_string in self.feature_dic.keys():
                if (prev_y, y) in self.feature_dic[feature_string].keys():
                    self.empirical_counts[self.feature_dic[feature_string][(prev_y, y)]] += 1
                else:
                    feature_id = self.num_features
                    self.feature_dic[feature_string][(prev_y, y)] = feature_id
                    self.empirical_counts[feature_id] += 1
                    self.num_features += 1
                if (-1, y) in self.feature_dic[feature_string].keys():
                    self.empirical_counts[self.feature_dic[feature_string][(-1, y)]] += 1
                else:
                    feature_id = self.num_features
                    self.feature_dic[feature_string][(-1, y)] = feature_id
                    self.empirical_counts[feature_id] += 1
                    self.num_features += 1
            else:
                self.feature_dic[feature_string] = dict()
                # Bigram feature
                feature_id = self.num_features
                self.feature_dic[feature_string][(prev_y, y)] = feature_id
                self.empirical_counts[feature_id] += 1
                self.num_features += 1
                # Unigram feature
                feature_id = self.num_features
                self.feature_dic[feature_string][(-1, y)] = feature_id
                self.empirical_counts[feature_id] += 1
                self.num_features += 1
    
    def scan(self, data):
        for sent in data:
            prev_y = START_LABEL_IDX
            for idx in range(len(sent)):
                try:
                    y_idx = self.label_dic(sent[idx][-1])
                except KeyError:
                    y_idx = len(self.label_dic)
                    self.label_dic[sent[idx][-1]] = y_idx
                    self.label_array.append(sent[idx][-1])
                self.add(prev_y, y, sent, idx)
                prev_y = y
                
        
    def _generate_all_feats_labels(self):
        
        for s in self.X:
            feats_list, labels_list,_,_ = self.get_sentence_feats_labels(s)
            self.feats.update(feats_list)
            self.labels.update(labels_list)
            
    def get_dataset_feats_labels():
        return self.feats, self.labels
    
    def feats2idx_func(self):
        for idx,f in enumerate(self.feats.keys()):
            self.feats2idx[f] = idx
            
    def labels2idx_func(self):
        labels = self.labels.keys()
        
        for idx, lbl in enumerate(labels):
            self.labels2idx[lbl] = idx
            
        length = len(self.labels2idx.keys())
        for idx, lbl in enumerate(labels):
            for idxx, lbll in enumerate(labels):
                self.labels2idx[lbl + SEP2 + lbll] = length+ idx*length + idxx
    
        length = len(self.labels2idx.keys())
        for idx, lbl in enumerate(labels):
            self.labels2idx[START_LABEL + SEP2 + lbl] = length + 2*idx
            self.labels2idx[lbl + SEP2 + STOP_LABEL] = length + 2*idx + 1                # include STOP_LABEL
    
    def featfunc_size(self):    
        self.feats_size, self.labels_size = len(self.feats.keys()), len(self.labels.keys())
        return self.feats_size, self.labels_size

### CRF

In [56]:
class LinearChainCRF():
    def __init__(self, data):
        self.data = data
        self.Feat = Feature(data)
        self.featfunc_matrix = None
        self.feats2idx=None
        self.labels2idx = None
        self.valid_featfunc_in_data = None
        self.labels_size, self.feats_size = 0, 0
        self.labels = None
        
    def init_params(self):
        self.Feat._generate_all_feats_labels()
        self.Feat.feats2idx_func()
        self.feats2idx = self.Feat.feats2idx
        self.Feat.labels2idx_func()                                                     # START_LABEL, included
        self.labels2idx = self.Feat.labels2idx
        self.labels = list(self.Feat.labels.keys())                                     # START_LABEL,  not included
        self.labels2idx_formal = {l:i for i,l in enumerate(self.labels)}
        self.feats_size, self.labels_size = self.Feat.featfunc_size()
        self.featfunc_matrix = np.zeros([len(self.feats2idx),len(self.labels2idx)])
        
    def empirical_expectation(self, batch_data):
        valid_funcs = Counter()
        emp_exp = 0
        for sent in batch_data:
            _,_,valid_func,_ = self.Feat.get_sentence_feats_labels(sent)
            valid_funcs.update(valid_func)
        
        for vf,c in valid_funcs.items():
            feat, lbl = vf.split(SEP1)
            emp_exp += self.featfunc_matrix[self.feats2idx[feat],self.labels2idx[lbl]] * c
        
        return emp_exp,valid_funcs
    
    def _forward_backward(self, sent):
        alpha0 = np.zeros(self.labels_size)
        beta0 = np.zeros(self.labels_size)
        alpha0[0],beta0[0] = 1,1
        
        sent_length = len(sent)
        
        sent_feats,_,_, featfunc_in_sent_position = self.Feat.get_sentence_feats_labels(sent)
        # M_n(y_{n-1},y{n}|x)
        M = []
        for n in range(sent_length+1):
            M_n = np.zeros([self.labels_size, self.labels_size])
            if n==0:
                feats_idx = [self.feats2idx[sent_feats[j]] for j in range(len(sent_feats))]
                for i in range(self.labels_size):
                    labels_idx = self.labels2idx[START_LABEL + SEP2 + self.labels[i]]
                    M_n[0,i] += np.exp(np.sum(self.featfunc_matrix[feats_idx,labels_idx]))
                M.append(M_n)
            elif n<sent_length:
                feats_idx = [self.feats2idx[sent_feats[j]] for j in range(len(sent_feats))]
                for k in range(self.labels_size):
                    for i in range(self.labels_size):
                        labels_idx = self.labels2idx[self.labels[k]+SEP2+self.labels[i]]
                        M_n[k,i] += np.exp(np.sum(self.featfunc_matrix[feats_idx,labels_idx]))
                M.append(M_n)
            else:
                feats_idx = [self.feats2idx[sent_feats[j]] for j in range(len(sent_feats))]
                for k in range(self.labels_size):
                    labels_idx = self.labels2idx[self.labels[k]+SEP2+ STOP_LABEL]
                    M_n[k,0] += np.exp(np.sum(self.featfunc_matrix[feats_idx,labels_idx]))
                M.append(M_n)
                
        alphas, betas = [alpha0],[beta0]
        prev_alpha, cur_beta = alpha0, beta0
        Z = np.eye(self.labels_size)
        
        for n in range(sent_length+1):
            cur_alpha = prev_alpha.dot(M[n])
            alphas.append(cur_alpha)
            prev_alpha = alphas[-1]
            Z = Z.dot(M[n])
        for n in range(sent_length,-1,-1):   
            prev_beta = M[n].dot(cur_beta)
            betas.insert(0,prev_beta)
            cur_beta = betas[0]

        
        norm_factor = Z[0,0]
        
        return M, np.array(alphas), np.array(betas), norm_factor, featfunc_in_sent_position
        
    def train(self, num_iters, batch_size, learning_rate, reg=10, do_reg=True):
        data_size = len(self.data)
        batch_size=data_size
        
        for i in range(num_iters):
            for b in range(0, data_size, batch_size):
                batch_data = self.data[b:b+batch_size]
                emp_exp, valid_funcs = self.empirical_expectation(batch_data)
                norm = 0
                gradients = np.zeros_like(self.featfunc_matrix)
                for sent in batch_data:
                    
                    M,alpha,beta,Z, featfunc_in_sent_position = self._forward_backward(sent)
                    norm += np.log(Z)
                    
                    for f,pos in featfunc_in_sent_position.items():
                        if len(pos) > 0:
                            feat, label = f.split(SEP1)
                            lbl = label.split(SEP2)
                            fidx = self.feats2idx[feat]
                            labels_idx = self.labels2idx[label]
                            if len(lbl) == 1:
                                for p in pos:
                                    gradients[fidx,labels_idx] -= alpha[p][labels_idx]*beta[p][labels_idx]/Z
                            if len(lbl) == 2:
                                lbl1, lbl2 = lbl[0],lbl[1]
                                if lbl1 == START_LABEL: # can only at p=0
                                    lbl2_idx = self.labels2idx_formal[lbl2]
                                    gradients[fidx, labels_idx] -= alpha[0][0]*M[0][0,lbl2_idx]*beta[1][lbl2_idx]/Z
                                elif lbl2 == STOP_LABEL:
                                    lbl1_idx = self.labels2idx_formal[lbl1]
                                    gradients[fidx, labels_idx] -= alpha[pos[0]][lbl1_idx]*M[pos[0]][lbl1_idx,0]*beta[pos[0]+1][0]/Z 
                                else:
                                    lbl1_idx = self.labels2idx_formal[lbl1]
                                    lbl2_idx = self.labels2idx_formal[lbl2]
                                    for p in pos:
                                        gradients[fidx,labels_idx] -= alpha[p][lbl1_idx]*M[p][lbl1_idx,lbl2_idx]*beta[p+1][lbl2_idx]/Z
                
                log_likelihood = -(emp_exp - norm) 
                
                if do_reg:
                    log_likelihood -= np.sum(self.featfunc_matrix**2)/(2*reg)
                    gradients -= self.featfunc_matrix/reg
                    
                for f,c in valid_funcs.items():
                    feat, label = f.split(SEP1)
                    fidx,lidx = self.feats2idx[feat],self.labels2idx[label]
                    gradients[fidx,lidx] += c
                    
                #update
                self.featfunc_matrix += learning_rate * gradients
                print(np.nonzero(self.featfunc_matrix)[0].size)
                print('log likelihood:', log_likelihood)
                print('-'*20)
    def test(self):
        pass
    def predict(self):
        pass

In [57]:
crf = LinearChainCRF(train_data)
crf.init_params()
crf.train(30,32,1e-4,10,do_reg=False)

31233
log likelihood: 5003.65269695053
--------------------
31299
log likelihood: 4923.562969063472
--------------------
31306
log likelihood: 4856.615751029051
--------------------


KeyboardInterrupt: 

In [58]:
import json

In [64]:
b=np.ones((3,3))
a={'s':b}
f = open(r'data\base_modell','w')
json.dump(a,f)

TypeError: Object of type ndarray is not JSON serializable

In [62]:
help(json)

Help on package json:

NAME
    json

DESCRIPTION
    JSON (JavaScript Object Notation) <http://json.org> is a subset of
    JavaScript syntax (ECMA-262 3rd edition) used as a lightweight data
    interchange format.
    
    :mod:`json` exposes an API familiar to users of the standard library
    :mod:`marshal` and :mod:`pickle` modules.  It is derived from a
    version of the externally maintained simplejson library.
    
    Encoding basic Python object hierarchies::
    
        >>> import json
        >>> json.dumps(['foo', {'bar': ('baz', None, 1.0, 2)}])
        '["foo", {"bar": ["baz", null, 1.0, 2]}]'
        >>> print(json.dumps("\"foo\bar"))
        "\"foo\bar"
        >>> print(json.dumps('\u1234'))
        "\u1234"
        >>> print(json.dumps('\\'))
        "\\"
        >>> print(json.dumps({"c": 0, "b": 0, "a": 0}, sort_keys=True))
        {"a": 0, "b": 0, "c": 0}
        >>> from io import StringIO
        >>> io = StringIO()
        >>> json.dump(['streaming API'], io